In [1]:
from dotenv import load_dotenv

load_dotenv()

True

## Creating subagents

In [3]:
from langchain.tools import tool

@tool
def square_root(x: float) -> float:
    """Calculate the square root of a number"""
    return x ** 0.5

@tool
def square(x: float) -> float:
    """Calculate the square of a number"""
    return x ** 2

In [6]:
from langchain.agents import create_agent
from langchain_ollama import ChatOllama

model = ChatOllama(model="lfm2.5-thinking")

# create subagents

subagent_1 = create_agent(
    model=model,
    tools=[square_root]
)

subagent_2 = create_agent(
    model=model,
    tools=[square]
)

## Calling subagents

In [7]:
from langchain.messages import HumanMessage
from langchain_ollama import ChatOllama

model = ChatOllama(model="lfm2.5-thinking")

@tool
def call_subagent_1(x: float) -> float:
    """Call subagent 1 in order to calculate the square root of a number"""
    response = subagent_1.invoke({"messages": [HumanMessage(content=f"Calculate the square root of {x}")]})
    return response["messages"][-1].content

@tool
def call_subagent_2(x: float) -> float:
    """Call subagent 2 in order to calculate the square of a number"""
    response = subagent_2.invoke({"messages": [HumanMessage(content=f"Calculate the square of {x}")]})
    return response["messages"][-1].content

## Creating the main agent

main_agent = create_agent(
    model=model,
    tools=[call_subagent_1, call_subagent_2],
    system_prompt="You are a helpful assistant who can call subagents to calculate the square root or square of a number.")

## Test

In [8]:
question = "What is the square root of 456?"

response = main_agent.invoke({"messages": [HumanMessage(content=question)]})

In [9]:
from pprint import pprint

pprint(response)

{'messages': [HumanMessage(content='What is the square root of 456?', additional_kwargs={}, response_metadata={}, id='24cd7a90-1808-404f-a6f5-75295d34b307'),
              AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'lfm2.5-thinking', 'created_at': '2026-02-20T09:16:21.166987Z', 'done': True, 'done_reason': 'stop', 'total_duration': 3697253416, 'load_duration': 479572000, 'prompt_eval_count': 168, 'prompt_eval_duration': 164681333, 'eval_count': 407, 'eval_duration': 2988035672, 'logprobs': None, 'model_name': 'lfm2.5-thinking', 'model_provider': 'ollama'}, id='lc_run--019c7a55-ecbb-7521-8919-cf0b6ca862fa-0', tool_calls=[{'name': 'call_subagent_1', 'args': {'x': 456}, 'id': '07a1df52-ca30-462d-afe5-7616f4df910f', 'type': 'tool_call'}], invalid_tool_calls=[], usage_metadata={'input_tokens': 168, 'output_tokens': 407, 'total_tokens': 575}),
              ToolMessage(content='The square root of 456.0 is approximately **21.354156504062622**. This value is derive